## Metaclasses

In [19]:
import torch
print(torch.__version__)
import functools
from numpy import array,ndarray
from itertools import starmap,dropwhile,takewhile,zip_longest
from copy import copy,deepcopy
from multiprocessing import Lock,Process,Queue,queues
from datetime import datetime
from contextlib import redirect_stdout,contextmanager
from typing import Iterable,Iterator,Generator,Callable,Sequence,List,Tuple,Union,Optional
import io,operator,sys,os,re,os,mimetypes,csv,itertools,json,shutil,glob,pickle,tarfile,collections

1.2.0


In [16]:
def is_iter(o):
    "Test whether `o` can be used in a `for` loop"
    #Rank 0 tensors in PyTorch are not really iterable
    return isinstance(o, (Iterable,Generator)) and getattr(o,'ndim',1)

In [14]:
def equals(a,b):
    "Compares `a` and `b` for equality; supports sublists, tensors and arrays too"
    if isinstance(a,type): return a==b
    if hasattr(a, '__array_eq__'): return a.__array_eq__(b)
    cmp = (np.array_equal if isinstance(a, ndarray   ) else
           operator.eq    if isinstance(a, (str,dict,set)) else
           all_equal      if is_iter(a) else
           operator.eq    if a.__eq__ != object.__eq__ else
           operator.eq)
    return cmp(a,b)

In [10]:
def test(a, b, cmp,cname=None):
    "`assert` that `cmp(a,b)`; display inputs and `cname or cmp.__name__` if it fails"
    if cname is None: cname=cmp.__name__
    assert cmp(a,b),f"{cname}:\n{a}\n{b}"

In [11]:
def test_eq(a,b):
    "`test` that `a==b`"
    test(a,b,equals, '==')

In [5]:
#export
class PrePostInitMeta(type):
    "A metaclass that calls optional `__pre_init__` and `__post_init__` methods"
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        def _pass(self, *args,**kwargs): pass
        for o in ('__init__', '__pre_init__', '__post_init__'):
            if not hasattr(x,o): setattr(x,o,_pass)
        old_init = x.__init__
        
        @functools.wraps(old_init)
        def _init(self,*args,**kwargs):
            self.__pre_init__()
            old_init(self, *args,**kwargs)
            self.__post_init__()
        setattr(x, '__init__', _init)
        return x

In [6]:
class _T(metaclass=PrePostInitMeta):
    def __pre_init__(self):  self.a  = 0; assert self.a==0
    def __init__(self):      self.a += 1; assert self.a==1
    def __post_init__(self): self.a += 1; assert self.a==2

t = _T()
t.a

2

In [7]:
#export
class NewChkMeta(PrePostInitMeta):
    "Metaclass to avoid recreating object passed to constructor (plus all `PrePostInitMeta` functionality)"
    def __new__(cls, name, bases, dct):
        x = super().__new__(cls, name, bases, dct)
        old_init,old_new = x.__init__,x.__new__

        @functools.wraps(old_init)
        def _new(cls, x=None, *args, **kwargs):
            if x is not None and isinstance(x,cls):
                x._newchk = 1
                return x
            res = old_new(cls)
            res._newchk = 0
            return res

        @functools.wraps(old_init)
        def _init(self,*args,**kwargs):
            if self._newchk: return
            old_init(self, *args, **kwargs)

        x.__init__,x.__new__ = _init,_new
        return x

In [8]:
class _T(metaclass=NewChkMeta):
    "Testing"
    def __init__(self, o=None): self.foo = getattr(o,'foo',0) + 1

class _T2():
    def __init__(self, o): self.foo = getattr(o,'foo',0) + 1

In [20]:
t = _T(1)
test_eq(t.foo,1)

In [21]:
t.foo

1

In [27]:
#export
class BypassNewMeta(type):
    "Metaclass: casts `x` to this class, initializing with `_new_meta` if available"
    def __call__(cls, x, *args, **kwargs):
        if hasattr(cls, '_new_meta'):
            print("True")
            x = cls._new_meta(x, *args, **kwargs)
        print(x,x.__class__,cls)
        if cls!=x.__class__: x.__class__ = cls
        return x

In [28]:
class T0: pass
class _T(T0, metaclass=BypassNewMeta): pass

In [29]:
t = T0()

In [30]:
t.a = 1

In [31]:
t2 = _T(t)

<__main__.T0 object at 0x120b227b8> <class '__main__.T0'> <class '__main__._T'>


In [32]:
test_eq(type(t2), _T)

## Decorators